# Error Analysis
We first set up the functions, and then do the analysis. 

## Setup

In [46]:
import torch
from preprocessing import preprocess_item, get_unique_tokens, token_to_index, setup_glove
from modules import FullModel, EmbeddingModule, CombinationModule


In [35]:
# setup glove
all_unique_toks = get_unique_tokens(None, None, None) # can be None because unique_tokens are loaded from pickle
glove_vectors, glove_vocab = setup_glove(all_unique_toks)

In [59]:
def build_example(premise, hypothesis, label):
    # make example
    example_text = {
        'premise': premise,
        'hypothesis': hypothesis,
        'label': label
    }
    # tokenize it
    example_tok = example_text.copy()
    example_tok = preprocess_item(example_tok)
    example_prep = token_to_index(example_tok, glove_vocab)
    
    prem_tok = torch.tensor(example_prep['premise']).cuda().unsqueeze(0)
    hyp_tok = torch.tensor(example_prep['hypothesis']).cuda().unsqueeze(0)
    lab_tok = torch.tensor(example_prep['label']).cuda().unsqueeze(0)
    
    return example_text, (prem_tok, hyp_tok, lab_tok)
    

premise = "Two men sitting in the sun"
hypothesis = "Nobody is sitting in the shade"
# 0: entailment, 1: neutral, 2: contradiction
label = 1 

example_text, example_prep = build_example(premise, hypothesis, label)
print(f'text: \n{example_text}')
print(f'\npreprocessed:\n{example_prep}')

text: 
{'premise': 'Two men sitting in the sun', 'hypothesis': 'Nobody is sitting in the shade', 'label': 1}

preprocessed:
(tensor([[34378, 19979, 29446, 16457, 32987, 31890]], device='cuda:0'), tensor([[21583, 17072, 29446, 16457, 32987, 28751]], device='cuda:0'), tensor([1], device='cuda:0'))


In [60]:
# initialize model
model = FullModel(
    EmbeddingModule(glove_vectors, all_unique_toks),
    None,
    CombinationModule(),
    None
).cuda()
model.load_model("checkpoints/pooledbilstm.pickle")


In [85]:
def print_prediction(pred, targ=None):
    d = ['Entailment', 'Neutral', 'Contradiction']
    s = f"Prediction:   {d[pred]}"
    if targ:
        s += f'\nGround truth: {d[targ]}'
    print(s)

## Analysis
First, let the model predict the first example. 

In [92]:
example_text

{'premise': 'Two men sitting in the sun',
 'hypothesis': 'Nobody is sitting in the shade',
 'label': 1}

In [89]:
# test the prediction
pred = model(example_prep).argmax()
print_prediction(pred, example_prep[2])

Prediction:   Entailment
Ground truth: Neutral


We can see here that ...


In [91]:
ex2, ex2_prep = build_example(
    "A man is walking a dog",
    "No cat is outside",
    1
)
pred2 = model(ex2_prep).argmax()
print_prediction(pred, ex2_prep[2])

Prediction:   Entailment
Ground truth: Neutral


We can see here that ...
